In [1]:
%run db.ipynb
from plpython_utils import plpython
schema = 'ds_training_wnv'
%read_sql SET search_path to {schema}

Query started at 09:48:35 AM EDT; Query executed in 0.00 m

# Random forest

## Create parameters table

In [2]:
%%read_sql
DROP TABLE IF EXISTS hyperparameters;
CREATE TABLE hyperparameters
AS
SELECT row_number() OVER () AS model_id, * 
FROM (SELECT generate_series(10,501,100) AS num_trees) a 
      CROSS JOIN (SELECT generate_series(1,101,10) AS max_depth) b
      CROSS JOIN (SELECT generate_series(1,101,10) AS max_features) c
-- DISTRIBUTE BY model_id;

Query started at 09:46:59 AM EDT; Query executed in 0.01 m

### Count the number of hyperparameters

In [3]:
%read_sql SELECT COUNT(*) FROM hyperparameters

Query started at 09:47:01 AM EDT; Query executed in 0.00 m

,count
0,605


# Evaluate multiple parameters

In [4]:
#TODO: trap dummy variable
features = ['id', 'longitude','latitude', 'year', 'month', 'day', 'day_of_year',\
 'species_CULEX ERRATICUS', 'species_CULEX PIPIENS', 'species_CULEX PIPIENS/RESTUANS',\
 'species_CULEX RESTUANS', 'species_CULEX SALINARIUS', 'species_CULEX TARSALIS',\
 'species_CULEX TERRITANS', 'avg', 'preciptotal7', 'tmin7', 'tavg7', 'tmax7']
feature_formatted = ['"{}"'.format(v) for v in features]  # select variables with spaces
feature_str = ','.join(feature_formatted)

In [64]:
%load_ext autoreload
%autoreload 2
from plpython_utils import plpython

# hello world
@plpython(conn, 'wnv', 'float')
def rf_train_eval(array1d,
                  column_names, # hello
                  dep1d,
                  id_col_name,
                  num_trees,
                  max_depth,
                  max_features):
    '''float[], text[], int[], text, int, int, int'''
    import numpy as np
    import pandas as pd
    
    from sklearn.cross_validation import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    
    num_rows = len(array1d)/len(column_names) 
    array2d = np.reshape(array1d, (num_rows, len(column_names)))
#     X = pd.DataFrame(array2d, columns = column_names)
#     X = X.drop(id_col_name, axis = 1)
#     y = dep1d
#     X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    
#     cl = RandomForestClassifier(n_estimators = 1000)
#     cl.fit(X_train,y_train, n_jobs=-1, num_estimators=num_trees, max_depth=mex_depth, max_features=max_features)    
    
#     y_prob = cl.pred_proba(X_test)[:,1]
#     return roc_auc_score(y_test, y_prob)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Successfully created function: wnv.rf_train_eval("array1d" float[], "column_names"  text[], "dep1d"  int[], "id_col_name"  text, "num_trees"  int, "max_depth"  int, "max_features"  int)


In [18]:
input_table = 'training_array'
hyperparam_table = 'hyperparameters'
output_table = 'hyperparameter_tuning_models'

In [19]:
%%read_sql
DROP TABLE IF EXISTS hyperparameter_tuning_models;
CREATE TABLE hyperparameter_tuning_models
AS
SELECT b.model_id, 
       rf_train_tune(features::float[],
                     array{feature_formatted}::text[],
                     wnv_present::int[], 'id',
                     b.num_trees,
                     b.max_depth,
                     b.max_features) AS model_obj
FROM {input_table} a, hyperparameters b;

Query started at 09:45:46 AM EDT; Query executed in 0.00 mQuery started at 09:45:46 AM EDT

DatabaseError: Execution failed on sql 'CREATE TABLE hyperparameter_tuning_models
AS
SELECT b.model_id, 
       rf_train_tune(features::float[],
                     array['"id"', '"longitude"', '"latitude"', '"year"', '"month"', '"day"', '"day_of_year"', '"species_CULEX ERRATICUS"', '"species_CULEX PIPIENS"', '"species_CULEX PIPIENS/RESTUANS"', '"species_CULEX RESTUANS"', '"species_CULEX SALINARIUS"', '"species_CULEX TARSALIS"', '"species_CULEX TERRITANS"', '"avg"', '"preciptotal7"', '"tmin7"', '"tavg7"', '"tmax7"']::text[],
                     wnv_present::int[], 'id',
                     b.num_trees,
                     b.max_depth,
                     b.max_features) AS model_obj
FROM training_array a, hyperparameters b;': relation "training_array" does not exist
LINE 10: FROM training_array a, hyperparameters b;
              ^


In [63]:
%read_sql SELECT * FROM {output_table}

Query started at 05:32:20 PM EDT
Query executed in 0.00 m


,model_id,model_obj
0,1,"[m, o, d, e, l]"
1,2,"[m, o, d, e, l]"
2,3,"[m, o, d, e, l]"
3,4,"[m, o, d, e, l]"
4,5,"[m, o, d, e, l]"
5,6,"[m, o, d, e, l]"
6,7,"[m, o, d, e, l]"
7,8,"[m, o, d, e, l]"
8,9,"[m, o, d, e, l]"
9,10,"[m, o, d, e, l]"


# Tune Logistic Regression

# MADlib